In [1]:
import requests
import urllib
import pandas as pd
from requests_html import HTML
from requests_html import HTMLSession
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------


Downloader>  q


True

In [2]:
class GoogleCrawler():
    
    def __init__(self):
        self.url = 'https://www.google.com/search?q='    
    #  URL 萃取 From Google Search上 , using 第三方套件
    #  https://python-googlesearch.readthedocs.io/en/latest/
    def google_url_search_byOpenSource(query,tbs='qdr:m',num=10):
        array_url = [url for url in search('tsmc', tbs='qdr:m' , num=10)]
        return array_url
    # 網路擷取器
    def get_source(self,url):
        try:
            session = HTMLSession()
            response = session.get(url)
            return response
        except requests.exceptions.RequestException as e:
            print(e)
    # URL 萃取 From Google Search上
    def scrape_google(self,query):

        response = self.get_source(self.url + query)
        links = list(response.html.absolute_links)
        google_domains = ('https://www.google.', 
                          'https://google.', 
                          'https://webcache.googleusercontent.', 
                          'http://webcache.googleusercontent.', 
                          'https://policies.google.',
                          'https://support.google.',
                          'https://maps.google.')

        for url in links[:]:
            if url.startswith(google_domains):
                links.remove(url)
        return links
    
# URL萃取器，有link之外，也有標題
#     qdr:h (past hour)
#     qdr:d (past day)
#     qdr:w (past week)
#     qdr:m (past month)
#     qdr:y (past year)
    def google_search(self,query,timeline='',page='0'):
        url = self.url + query + '&tbs={timeline}&start={page}'.format(timeline=timeline,page=page)
        print('[Check][URL] URL : {url}'.format(url=url))
        response = self.get_source(self.url + query)
        return self.parse_googleResults(response)
    # Google Search Result Parsing
    def parse_googleResults(self,response):

        css_identifier_result = "tF2Cxc"
        css_identifier_title = "h3"
        css_identifier_link = "yuRUbf"
        css_identifier_text = "VwiC3b"
        soup = BeautifulSoup(response.text, 'html.parser')
        results = soup.findAll("div", {"class": css_identifier_result})
        output = []
        for result in results:
            item = {
                'title': result.find(css_identifier_title).get_text(),
                'link': result.find("div", {"class": css_identifier_link}).find(href=True)['href'],
                'text': result.find("div", {"class": css_identifier_text}).get_text()
            }
            output.append(item)
        return output
    
    # 網頁解析器
    def html_parser(self,htmlText):
        soup = BeautifulSoup(htmlText, 'html.parser')
        return soup
    # 解析後，取<p>文字
    def html_getText(self,soup):
        orignal_text = ''
        for el in soup.find_all('p'):
            orignal_text += ''.join(el.find_all(text=True))
        return orignal_text
    
    def word_count(self, text):
        counts = dict()
        stop_words = set(stopwords.words('english'))
        words = word_tokenize(text)
        #words = text.replace(',','').split()
        for word in words:
            if word not in stop_words:
                if word in counts:
                    counts[word] += 1
                else:
                    counts[word] = 1
        return counts
    def word_count_ch(self, text):
        counts = dict()
        import jieba
        jieba.enable_paddle()# 启动paddle模式。 0.40版之后开始支持，早期版本不支持
        words = jieba.cut(text, cut_all=False)
        print(", ".join(words))
        for word in words:
            if word in counts:
                counts[word] += 1
            else:
                counts[word] = 1
        return counts
    def get_wordcount_json(self,whitelist , dict_data):
        data_array = []
        for i in whitelist:
            json_data = {
                'Date' : 'Week1',
                'Company' : i , 
                'Count' : dict_data[i]
            }
            data_array.append(json_data)
        return data_array
    def jsonarray_toexcel(self,data_array, path = 'result.xlsx'):
        df = pd.DataFrame(data=data_array)
        df.to_excel(path , index=False)
        return

## 程式使用情境
### 1. 用程式去呼叫Google網頁伺服器，然後將顯示出來的URL，抓出來。 (支援換頁及限縮搜尋時間)
### 2. 對我們要的URL，進行萃取，取得文字內容。
### 3. 將文字內容，進行word count計算。
### 4. 將word count結果儲存到Excel，以利分析之使用。

### 1. 在Google上輸入關鍵字，然後將顯示出來的URL，抓出來。 (支援換頁及限縮收尋時間)

#### 使用google_search 這支Function
- 參數使用 timeline : 搜尋時間 => 參數可以參考qdr:h (past hour), qdr:d (past day),qdr:w (past week),qdr:m (past month),qdr:y (past year)
- 參數使用 page : 換頁
```
def google_search(self,query,timeline='',page='0'):
    url = self.url + query + '&tbs={timeline}&start={page}'.format(timeline=timeline,page=page)
    print('[Check][URL] URL : {url}'.format(url=url))
    response = self.get_source(self.url + query)
    return self.parse_googleResults(response)
```
    
####  這邊用到get_source及parse_googleResults，兩支Function，為了解析Google Search網路資源，並將URL/標題/內文，找出來。
```
def get_source(self,url):
    try:
        session = HTMLSession()
        response = session.get(url)
        return response
    except requests.exceptions.RequestException as e:
        print(e)
def parse_googleResults(self,response):
        css_identifier_result = "tF2Cxc"
        css_identifier_title = "h3"
        css_identifier_link = "yuRUbf"
        css_identifier_text = "VwiC3b"
        soup = BeautifulSoup(response.text, 'html.parser')
        results = soup.findAll("div", {"class": css_identifier_result})
        output = []
        for result in results:
            item = {
                'title': result.find(css_identifier_title).get_text(),
                'link': result.find("div", {"class": css_identifier_link}).find(href=True)['href'],
                'text': result.find("div", {"class": css_identifier_text}).get_text()
            }
            output.append(item)
        return output ```

In [3]:
query = "TSMC ASML"
crawler = GoogleCrawler()
results = crawler.google_search(query , 'qdr:w' , '10')
print(results[:3])

[Check][URL] URL : https://www.google.com/search?q=TSMC ASML&tbs=qdr:w&start=10
[{'title': '台積電、三星真是ASML 大股東，採購EUV 有特別禮遇？', 'link': 'https://technews.tw/2022/03/29/are-intel-tsmc-and-samsung-really-the-major-shareholders-of-asml/', 'text': '2022年3月29日 — 目前能生產7 奈米以下晶片的極紫外光(EUV) 曝光機，只有荷蘭商艾司摩爾(ASML)，ASML 可說是掌握全球晶片先進製程的關鍵。ASML 的EUV 曝光機通常優先供\xa0...'}, {'title': '英特爾突買ASML最強機台！台積電慘了？專家驚爆真相', 'link': 'https://www.chinatimes.com/realtimenews/20220124002784-260410', 'text': '2022年1月24日 — 半導體生產設備大廠艾司摩爾（ASML）上周宣布，英特爾成為首家下訂最先進 ... 南韓經濟日報報導，英特爾訂購EUV曝光設備比台積電3奈米製程的EUV設備更\xa0...'}, {'title': '只剩台積電、三星可以繼續玩下去...兩巨頭幾乎包辦了ASML ...', 'link': 'https://tw.news.yahoo.com/%E5%8F%AA%E5%89%A9%E5%8F%B0%E7%A9%8D%E9%9B%BB-%E4%B8%89%E6%98%9F%E5%8F%AF%E4%BB%A5%E7%B9%BC%E7%BA%8C%E7%8E%A9%E4%B8%8B%E5%8E%BB-%E5%85%A9%E5%B7%A8%E9%A0%AD%E5%B9%BE%E4%B9%8E%E5%8C%85%E8%BE%A6%E4%BA%86asml%E6%9B%9D%E5%85%89%E6%A9%9F%E5%85%A8%E9%83%A8%E7%94%A2%E8%83%BD-051327897.html', 'text': '2022年1月17日 — 雖著全球對於晶片需求增加，各類半導體生產設備的需求也隨之上升。(圖片來源/

### 2. 對我們要的URL，進行萃取，取得文字內容。
- 使用get_source 這支Function，取得網路資源後，
- 使用html_parser 這支Function，將網路資源進行解析。
- 使用html_getText 這支Function，將我要的區塊p tag的文字取出來。
```
def get_source(self,url):
    try:
        session = HTMLSession()
        response = session.get(url)
        return response
    except requests.exceptions.RequestException as e:
        print(e)
def html_parser(self,htmlText):
    soup = BeautifulSoup(htmlText, 'html.parser')
    return soup
def html_getText(self,soup):
    orignal_text = ''
    for el in soup.find_all('p'):
        orignal_text += ''.join(el.find_all(text=True))
    return orignal_text```

In [4]:
Target_URL = 'https://taipeitimes.com/News/biz/archives/2022/01/20/2003771688'
response = crawler.get_source(Target_URL)
soup = crawler.html_parser(response.text)
orignal_text = crawler.html_getText(soup)
print(orignal_text[:100])

Intel Corp yesterday said it has placed its first order with ASML Holding NV to purchase the semicon


### 3. 將文字內容，進行word count計算
#### 使用word_count function，將一篇文章內容，進行各文字的count個數
```
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
def word_count(self, text):
        counts = dict()
        stop_words = set(stopwords.words('english'))
        words = word_tokenize(text)
        #words = text.replace(',','').split()
        for word in words:
            if word not in stop_words:
                if word in counts:
                    counts[word] += 1
                else:
                    counts[word] = 1
        return counts ```

In [5]:
result_wordcount = crawler.word_count(orignal_text)
result_wordcount

{'Intel': 5,
 'Corp': 1,
 'yesterday': 5,
 'said': 15,
 'placed': 1,
 'first': 7,
 'order': 2,
 'ASML': 7,
 'Holding': 1,
 'NV': 1,
 'purchase': 3,
 'semiconductor': 5,
 'industry': 5,
 '’': 15,
 'TWINSCAN': 5,
 'EXE': 1,
 ':': 7,
 '5200': 1,
 'system': 8,
 ',': 47,
 'US': 2,
 'chip': 4,
 'giant': 2,
 'aims': 2,
 'compete': 1,
 'Taiwan': 4,
 'Semiconductor': 1,
 'Manufacturing': 1,
 'Co': 2,
 '(': 8,
 'TSMC': 4,
 '台積電': 1,
 ')': 8,
 'advancing': 1,
 '2-nanometer': 4,
 'process': 1,
 'technology': 6,
 '.': 22,
 'The': 10,
 'Dutch': 1,
 'equipment': 2,
 'maker': 1,
 'EXE:5200': 2,
 'extreme': 1,
 'ultraviolet': 1,
 'EUV': 4,
 'high-volume': 1,
 'production': 5,
 'high': 1,
 'numerical': 1,
 'aperture': 1,
 'NA': 1,
 'produce': 2,
 '220': 1,
 'wafers': 2,
 'per': 1,
 'hour': 1,
 '150': 1,
 'previous': 1,
 'generation': 1,
 'EXE:5000': 2,
 'handle': 1,
 'launch': 1,
 'new': 4,
 '2024': 1,
 'Photo': 1,
 'ReutersASML': 1,
 'president': 2,
 'chief': 2,
 'officer': 2,
 'Martin': 1,
 'van': 1,


### 4. 將聲量結果儲存到Excel
#### 使用get_wordcount_json function濾掉不要的word count dict, 取自己要的(whitelist)
#### 同時利用jsonarray_toexcel function將結果，落地於Excel，以做聲量分析
```
def get_wordcount_json(self,whitelist , dict_data):
        data_array = []
        for i in whitelist:
            json_data = {
                'Date' : 'Week1',
                'Company' : i , 
                'Count' : dict_data[i]
            }
            data_array.append(json_data)
        return data_array
def jsonarray_toexcel(self,data_array):
    df = pd.DataFrame(data=data_array)
    df.to_excel('result.xlsx' , index=False)
    return
```

In [6]:
whitelist = ['ASML' , 'Intel']
end_result = crawler.get_wordcount_json(whitelist , result_wordcount)
print(end_result)
crawler.jsonarray_toexcel(end_result)
print('Excel is OK')

[{'Date': 'Week1', 'Company': 'ASML', 'Count': 7}, {'Date': 'Week1', 'Company': 'Intel', 'Count': 5}]
Excel is OK


In [10]:
for i, r in enumerate(results):
    Target_URL = r['link']
    response = crawler.get_source(Target_URL)
    soup = crawler.html_parser(response.text)
    orignal_text = crawler.html_getText(soup)
#     print(orignal_text[:100])
    result_wordcount = word_count_sp(orignal_text)
    whitelist = []#['ASML', 'Intel']
    end_result = crawler.get_wordcount_json(whitelist , result_wordcount)
    print(end_result)
    path = 'result_' + str(i).zfill(3) + '.xlsx'
    print(path)
    crawler.jsonarray_toexcel(end_result, path)
    print('Excel is OK', str(i).zfill(3))

Paddle enabled successfully......


目前, 能生產,  , 7,  , 奈米, 以下, 晶片, 的極, 紫外光,  , (, EUV, ),  , 曝光, 機, ，, 只有, 荷蘭商, 艾司, 摩爾,  , (, ASML, ), ，, ASML,  , 可說, 是, 掌握, 全球, 晶片, 先, 進, 製, 程, 的, 關, 鍵, 。, ASML,  , 的,  , EUV,  , 曝光, 機, 通常, 優先供, 貨給, 台積電, 、, 三星, 、, 英特, 爾, 三家, ，, 其他, 要, 買, 都, 要, 排, 在, 三家, 之, 後, 。, 但, 為, 什麼,  , ASML,  , 優先, 賣給, 這, 三家, ？, 首先, 當然, 是, 三家, 都, 是, 大, 客戶, ，, 因此, 有, 特別, 待遇, ，, 其次, 為, 外界, 都, 認為, 這, 三家, 是,  , ASML,  , 大股, 東, ，, 當然, 先照, 顧大, 股東, 。, 但台, 積電, 、, 三星, 、, 英特, 爾, 真是,  , ASML,  , 大股, 東, 嗎, ？, 故事, 得, 從,  , ASML,  , 歷史, 談起, 。,  , ASML,  , 歷史, 可追溯, 到,  , 1984,  , 年, ，, 荷蘭, 飛利浦, （, Philips, ）, 與, 半導體, 設, 備, 製, 造, 商,  , ASM,  , International,  , (, ASMI, ),  , 各出資,  , 210,  , 萬元, 創立,  , ASML, ，, 雙方, 各, 持股,  , 50%, ，, 但, 飛利浦, 於,  , ASML,  , 經營, 困難, 時, 又, 增資, ，, 持股, 比例, 上升, 。, 1995,  , 年,  , ASML,  , 正式, 上市, ，, 飛利浦, 連續, 分幾次, 處分,  , ASML,  , 股票, ，, 故飛利浦, 與,  , ASML,  , 現在, 沒有, 任何, 關, 係, 。, 之, 後,  , ASML,  , 迎接, 轉捩點, ，, 在, 台積, 電建議, 下, ，, 合作, 研發, 浸潤式, 曝光, 機, ，, 使,  , ASML,  , 突飛, 猛進, ，, 市占率, 超過, 老牌, 曝光, 機大廠, 日本,  , Canon

Paddle enabled successfully......


半導體, 生產, 設備, 大廠, 艾司, 摩爾, （, ASML, ）, 上周, 宣布, ，, 英特, 爾成, 為, 首家, 下訂, 最先, 進, 的, 高數值, 孔徑, （, High, -, NA, ）, 極, 紫外光, （, EUV, ）, 微影, 設備, 的, 客戶, 。, 雙方, 突大動作, 宣布, 合作, 引起, 市場, 熱議, ，, 韓媒則, 認為, 英特, 爾, 此舉, 是, 想要, 降低, 市場, 對, 其晶圓, 代工, 技術, 的, 質疑, 。, 南, 韓經濟, 日報, 報導, ，, 英特, 爾訂購, EUV, 曝光, 設備, 比台, 積電, 3, 奈, 米, 製, 程, 的, EUV, 設備, 更, 為, 先進, ，, 不少, 專家, 認為, ，, 英特, 爾, 可能, 會, 透過, 這些, 設備, 生產, 2, 奈, 米, 製, 程, 晶片, ，, 英特, 爾, 也, 在, 聲明, 中, 表示, ，, 新一代, 的, EUV, 機台, 可, 使晶, 圓, 製, 造, 速率, 每小時, 將超過,  , 200, 片, ，, 是, 發展, 先, 進, 製, 程, 的, 重要, 一環, 。, 英特, 爾, 與, 艾司, 摩爾大動, 作公開, 雙方, 合作, 計畫, ，, 報導, 引述, 市場, 人士, 意見, 指出, ，, 英特, 爾, 在, 先, 進, 製, 程, 的, 腳, 步, 並, 不, 穩, 定, ，, 突然, 宣布, 合作, 關, 係, ，, 目的, 是, 減少, 市場, 對, 英特, 爾技術, 質疑, 的, 壓力, ，, 但, 最終, 是否, 能夠, 如, 英特, 爾, 所願還, 有待, 觀察, 。, 此外, ，, 由, 於, 英特, 爾成, 為, 艾司, 摩爾, 最先, 進, 製, 造, 設, 備, 的, 首位, 客戶, ，, 投資, 研究, 機構, Semiconductor,  , Advisors, 分析, 師, Robert,  , Maire, 認為, ，, 如果, 英特, 爾, 可以, 搶先, 採用, 這些, 工具, ，, 可能, 是, 在, 摩爾, 定律, 的, 競賽, 當中, ，, 有, 機會, 超車, 台積電, 的, 好, 機會, 。, 中時, 新聞網, 對, 留言, 系統, 使用者, 發布, 的, 文字, 、, 

Paddle enabled successfully......


雖著, 全球, 對, 於, 晶片, 需求, 增加, ，, 各類, 半導體, 生產, 設備, 的, 需求, 也, 隨之, 上升, 。, (, 圖片, 來源, /, ASML,  , 官網, ), 台積電, 宣布, 今年, 的, 資本, 支出, 將高達, 400, 到, 440, 億, 美元, 歷史, 新高, ，, 龐大, 的, 資本, 支出, 讓, 相關, 半導體, 設備, 廠商, 吃, 下, 大補丸, 。, 蔡總統, 昨, （, 17, ）, 日接, 見, 「, 社團, 法人, 台灣, 電子, 設備, 協會, 」, 理事, 長, 等, 幹部, 。, 她, 指出, ，, 台灣, 半導, 體產業, 居, 世界, 領先, 地位, ，, 要, 如何, 把握, 台灣, 的, 半導, 體產業, 蓬勃, 發展, 的, 時機, ，, 進, 一步, 推動, 半導體, 設備, 國產化, ，, 是, 政府, 的, 政策, 、, 未來, 要, 努力, 的, 方向, ，, 也, 是, 協會, 重要, 的, 目標, 。, 蔡總統, 說, ，, 從, 台灣, 半導, 體產業, 整體, 投資來, 看, ，, 去年, 台灣, 半導體, 晶圓廠, 投資約, 新, 台幣, 1, 兆元, ，, 設備, 投資, 占, 七成, 、, 約, 7, ,, 000, 億元, ，, 其中, 有, 6, ,, 000, 億元, 是, 採購, 國外, 設備, ，, 這顯示, 國內, 設備, 業者, 還有, 很大, 的, 成長, 空間, 。, 蔡總統, 強調, ，, 未來將, 把, 台灣, 打, 造成, 為, 「, 亞洲, 高, 階, 製, 造, 中心, 」, 和, 「, 半導體, 先, 進, 製, 程, 中心, 」, 。, 去年, 台灣, 對外, 採購, 了, 6, 千億元, 的, 半導體, 設備, ，, 主要, 是, 跟, 哪些, 國際大廠, 採購, 呢, ？, 以下, 為, 周國華, 教授, 的, 專文, 分析, ：, 台灣, 有, 全球, 最大, 規模, 的, 晶圓, 代工, 產業, 鏈, ，, 以及, 南亞科, 、, 華邦電, 、, 旺宏, .., 等, 記憶, 體, 製, 造, 商, ，, 加上, 日, 月光, 及, 矽品, .., 等, 封測廠, ，, 所以, 對, 各類, 型, 的, 半導體, 設備, 有,

Paddle enabled successfully......



, ?, 
, 人氣, 
, 現正, 熱映, 中熱門, 文章, 熱門, 分享, 
, 	, 	, 	, 	, 	, 	, 	, ASML, 今年, 第一季, 財測, 低, 於, 市場, 預期, 。, （, 圖片, 來源, ：, ASML, ）, 
, 	, 	, 	, 	, 	, 	, 全球, 半導體, 微影, 技術, 及, 曝光, 機生, 產廠, 商艾司, 摩爾, （, ASML, ）, 公布, 去年, 財報, 與, 今年, 展望, ，, 但, 相關, 數據, 啟人疑, 竇, ，, 知名, 半導體, 分析, 師陸行, 之, 直言, 「, ASML, 把, 投資人, 當白, 癡, 」, ，, 今年, 第一季, 營收, 認列, 過程, 怎麼, 解釋, 都, 很, 奇怪, ，, 全年, 營, 收成, 長, 目標, 保守, ，, 也, 和, 台, 積電, 大幅, 增加, 的, 資本, 支出, 兜, 不, 起來, 。, [, 啟動, LINE, 推播, ],  , 每日, 重大, 新聞, 通知, ASML, 週三, （, 19, 日, ）, 公布, ，, 2021, 年, 第四季, 實現, 凈, 銷售額, 49.86, 億歐元, ，, 凈, 利潤, 17.74, 億歐元, ，, 毛利率, 54.2%, ，, 雖然, 營收, 不及, 市場, 預期, 的, 51, 億歐元, ，, 但利潤, 超乎, 分析, 師預期, 的, 15.1, 億歐元, 。, 展望, 今年, 第一季, ，, ASML, 預計, 因為, 加速, 出貨, 流程, ，, 直接, 到, 客戶, 工廠, 測試驗, 收, ，, 延遲, 認列將, 使, 營收, 下滑, 至, 33, ～, 35, 億歐元, ，, 季減, 30%, ～, 34%, 。, （, 延伸, 閱讀, ：, 「, 撞, 到, 這輛, 車賠, 30, 億, ！, 」, 台積, 電背, 後, 的, 靈魂, 軍火商, ，, 年花, 600, 萬養, 一名, 工程, 師, ）, 針對, ASML, 第一季, 財測, 大幅, 低, 於, 市場, 預期, ，, 陸行, 之, 今日, 在, 臉書上, 提出, 五大, 重點, 解讀, ，, 首先, 是, 公司, 解釋, 因為, 營收, 認列, 方式, 改變, ，, ASML, 這端, 取消, 驗證, 程序, 快速

Paddle enabled successfully......


Please,  , stand,  , by, ,,  , while,  , we,  , are,  , checking,  , your,  , browser, ..., Redirecting, ..., Please,  , enable,  , Cookies,  , and,  , reload,  , the,  , page, ., Completing,  , the,  , CAPTCHA,  , proves,  , you,  , are,  , a,  , human,  , and,  , gives,  , you,  , temporary,  , access,  , to,  , the,  , web,  , property, ., If,  , you,  , are,  , on,  , a,  , personal,  , connection, ,,  , like,  , at,  , home, ,,  , you,  , can,  , run,  , an,  , anti, -, virus,  , scan,  , on,  , your,  , device,  , to,  , make,  , sure,  , it,  , is,  , not,  , infected,  , with,  , malware, ., If,  , you,  , are,  , at,  , an,  , office,  , or,  , shared,  , network, ,,  , you,  , can,  , ask,  , the,  , network,  , administrator,  , to,  , run,  , a,  , scan,  , across,  , the,  , network,  , looking,  , for,  , misconfigured,  , or,  , infected,  , devices, ., 
, Cloudflare,  , Ray,  , ID, :,  , 7044f1556bac6a9c, 
, •, 
, Your,  , IP, :,  , 140.113, ., 24.138, 
, •, 
, Performa

In [8]:
def word_count_sp(text):
    counts = dict()
    import jieba
    jieba.enable_paddle()# 启动paddle模式。 0.40版之后开始支持，早期版本不支持
    words = jieba.cut(text, cut_all=False)
    print(", ".join(words))
    for word in words:
        if word in counts:
            counts[word] += 1
        else:
            counts[word] = 1
    return counts